In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# get test data
df_test = pd.read_csv("../input/test.csv")

# get train data
df_train = pd.read_csv("../input/train.csv")

# get gender submission

df_gender = pd.read_csv("../input/gender_submission.csv")

In [ ]:
# check df_train
df_train.sample(10)

In [ ]:
# info
df_train.info()

In [ ]:
# summary statistic
df_train.describe()

In [ ]:
# shape
df_train.shape

In [ ]:
# check duplication
df_train.duplicated().sum()

In [ ]:
# check missing value
df_train.isna().sum()

In [ ]:
# hist
df_train_clean.hist(figsize=[7,8]);

In [ ]:
# check max fare as it seems outlier
df_train["Fare"].sort_values(ascending = False).head()

In [ ]:
# check from which class the outliers come
df_train[df_train["Fare"] >500].Pclass

-  I will leave the outliers as they purchased 1 class thickets which should be expensive.

# Quality issue

- passenger id is int data type
- Pclass is int data type
- sex is string
- cabin is string
- embark is string
- missing value in Age, Embarked and especially Cabin


# Tidiness issue
- title is together with name


## Missing value
## Definition
> Missing age data need to be filled. Check if there any coorelation between age and some features to decide how to fill in.p

In [ ]:
# plot age, fare and pclass
g = sns.FacetGrid(data = df_train, hue = "Pclass", size = 5)
g.map(plt.scatter, "Age", "Fare", alpha = 0.7)
g.set(xlim =[0,90], ylim = [0, 200])
g.add_legend()
plt.title("Relationship among age, ticket fare and passenger class");

In [ ]:
# check how many missing age in each Pclass
df_train[df_train.Age.isnull()].Pclass.value_counts()

In [ ]:
# visualize with a SD line
sns.barplot(data = df_train, x = "Pclass", y = "Age", ci = "sd")

In [ ]:
# check distribution
sns.boxplot(data = df_train, x = "Pclass", y = "Age")

> I will take a mean of age in each Pclass and replace respectively.

In [ ]:
# get mean for each Pclass
first_mean, second_mean,third_mean = df_train.groupby("Pclass").Age.mean()

# replace nan with respective mean
df_train.Age = df_train.loc[df_train["Pclass"] == 1, "Age"].fillna(first_mean, inplace = True)
df_train.Age = df_train.loc[df_train["Pclass"] == 2, "Age"].fillna(second_mean, inplace = True)
df_train.Age = df_train.loc[df_train["Pclass"] == 3, "Age"].fillna(third_mean, inplace = True)

In [ ]:
# replace na with respective mean
age_na_1 = df_train[(df_train["Pclass"]== 1) & (df_train["Age"].isna())].index
df_train.loc[age_na_1,"Age"] = first_mean

age_na_2 = df_train[(df_train["Pclass"]== 2) & (df_train["Age"].isna())].index
df_train.loc[age_na_2,"Age"] = second_mean

age_na_3 = df_train[(df_train["Pclass"]== 3) & (df_train["Age"].isna())].index
df_train.loc[age_na_3,"Age"] = third_mean


> ## Test

In [ ]:
# check results
df_train.info()

## Definition

fill in missing values in the Embarked column.
* Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)

In [ ]:
# check 
df_train[df_train["Embarked"].isna()]

In [ ]:
# check frequency
sns.countplot(data = df_train, x = "Embarked", hue = "Pclass")

In [ ]:
# replace with the most frequent one which is S
embarked_na = df_train[df_train["Embarked"].isna()].index
df_train.loc[embarked_na, "Embarked"] = "S"

## Test

In [ ]:
# check
df_train.info()

In [ ]:
# check corralation NUMERICAL
sns.heatmap(df_train[["Survived","SibSp", "Parch","Fare","Pclass","Age"]].corr(), annot = True);

- Pclass & Survived, Fare & Survived, Pclass & Fare seem to have some correlations.

In [ ]:
# plot
plt.figure(figsize = [20,5])
plt.subplot(1,2,1)
sns.violinplot(data = df_train, x = "Pclass", y = "Fare", inner = "quartile")
# set y lim to max 200 as there are outliers
plt.ylim(0,200)
plt.title("Ticket price and passenger class")

# plpot2
plt.subplot(1,2,2)
sns.barplot(data = df_train, x = "Pclass", y = "Survived" )
plt.title("Survival posibility and passenger class");


# Clean

## Definition
Titles and name are in the same column. A new title column is created through a split function "," and " ". 

In [ ]:
# create a copy
df_train_clean = df_train.copy()

In [ ]:
# extract titles
df_train_clean["Title"] = df_train_clean["Name"].str.split(",").str[1].str.split(".").str[0].str.strip()

In [ ]:
# check unique titles
df_train_clean.Title.unique()

In [ ]:
# check unique titles
df_train_clean.Title.value_counts()

- Mr. = title for men   ----> man
- Miss. = title for unmarried women ----> woman
- Mrs. = title for married or widowed women ---> woman
- Master. = honorific for boys and young men ---> special
- Dr. = academic title ---> Special
- Rev. = The Reverend is an honorific style most often placed before the names of Christian clergy and ministers.(https://en.wikipedia.org/wiki/The_Reverend) ---> special
- Col. = honorary title Colonel ---> special
- Major. = military rank ----> military
- Mlle. = Mademoiselle is a French familiar title, abbreviated Mlle, traditionally given to an unmarried woman. (wikipedia) ---> woman
- Capt. = title for the commander of a military unit ---> military
- Ms. = title for unmarried or unknown status women ---> woman
- Sir. = The title of "Sir" is given to anyone awarded knighthood by the Queen or a member of the royal family (https://www.reference.com/world-view/title-sir-given-934711913b5cc171) ---> special
- Countess. = Countess is the Title taken by the wife of an Earl or Count ---> special
- Lady. = The honorific prefix of "Lady" is used for the daughters of dukes, marquesses and earls. (wikipedia) ---> special
- Jonkheer = honorific in the Low Countries denoting the lowest rank within the nobility (wikipedia) ---> special
- Don. = an honorific prefix primarly used in Italy, Spain, Portugal, Latin America, and the Philippines. (wikipedia) ---> special
- Mme. = Madam in French ---> woman

- Man, Woman, Military and honorific??











In [ ]:
# replace titles
# man, woman, military and special
df_train_clean["Title"].replace({"Mr":"man",
                                "Miss":"woman",
                                "Mrs":"woman",
                                "Master":"special",
                                "Dr":"special",
                                "Rev":"special",
                                 "Mlle":"woman",
                                "Col":"special",
                                "Major":"special",
                                "Sir":"special",
                                 "Mme":"woman",
                                 "Ms":"woman",
                                 "Capt":"special",
                                 "Don":"special",
                                 "Lady":"special",
                                 "Jonkheer":"special",
                                 "the Countess":"special"
                                },
                                 inplace = True)

In [ ]:
# check group mean
df_train_clean.groupby("Title").mean().Survived

In [ ]:
sns.countplot(data = df_train_clean, x = "Title", hue = "Survived")


# Test

In [ ]:
# check
df_train_clean.sample(10)

In [ ]:
# heat map
corr = df_train_clean.corr()

plt.figure(figsize=[10,7])
sns.heatmap(corr, annot = True, fmt = ".2f",cmap = "YlGnBu_r")

Correlationb
- Pclass and Fare
- Age and Pclass
- Survived and Pclass
- Survived and age
- Survived and Fare

In [ ]:
# check survival ratio for classes
sns.countplot(data = df_train_clean, x = df_train["Pclass"],hue = "Survived")

In [ ]:

g = sns.catplot(data = df_train_clean, x = "Pclass",col = "Survived", kind = "count", hue = "Sex" )